# Collect additional data

In [123]:
import pandas as pd
import csv

# Load the data to understand its structure and contents
file_path = '../data/CANIS_PRC_state_media_on_social_media_platforms-2023-11-03.xlsx'
data = pd.read_excel(file_path)

# Display the first few rows of the dataframe and summary information
data_info = data.info()
first_rows = data.head()

data_info


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758 entries, 0 to 757
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Name (English)           758 non-null    object 
 1   Name (Chinese)           319 non-null    object 
 2   Region of Focus          758 non-null    object 
 3   Language                 758 non-null    object 
 4   Entity owner (English)   758 non-null    object 
 5   Entity owner (Chinese)   755 non-null    object 
 6   Parent entity (English)  758 non-null    object 
 7   Parent entity (Chinese)  751 non-null    object 
 8   X (Twitter) handle       573 non-null    object 
 9   X (Twitter) URL          573 non-null    object 
 10  X (Twitter) Follower #   573 non-null    float64
 11  Facebook page            326 non-null    object 
 12  Facebook URL             326 non-null    object 
 13  Facebook Follower #      326 non-null    float64
 14  Instragram page          1

In [124]:
filtered_data = data[data['X (Twitter) handle'].notnull()]['X (Twitter) handle'] 
filtered_data.tolist()




['_bubblyabby_',
 '_cultureexpress',
 'ACSF1919',
 'afghanchina',
 'AlexYsalex17',
 'Amazing_Yunnan',
 'Amb_ChenXu',
 'Amb_LiSong',
 'Amb_LuKang',
 'Amb_Yiming',
 'Amb_ZhuJing',
 'AmbassadeChine',
 'Ambassador_Liu',
 'AmbassadorHuang',
 'AmbassadorLei',
 'AmbChangHua',
 'AmbChenBo',
 'AmbChenWeiqing',
 'AmbChineBurundi',
 'AmbChineCmr',
 'AmbCHINEenRDC',
 'ambchinetchad',
 'AmbCina',
 'AmbCuiTiankai',
 'AmbDongXiaojun',
 'AmbFangQiu',
 'AmbJiaGuide',
 'AmbKongXuanyou',
 'AmbLiaoLiqiang',
 'AmbLiuGuangYuan',
 'AmbLiuQuan',
 'AmbLiuXiaoMing',
 'AmbNong',
 'AmbQinGang',
 'AmbWuJianghao',
 'AmbXieFeng',
 'AmbZhengZeguang',
 'AmbZhouJian',
 'aMEDIAchallenge',
 'AndyBxxx',
 'AnnaGE_',
 'anosiwang',
 'are_china',
 'assignasia',
 'ayishaelseenya',
 'back_XiaoDiDi',
 'BeautyJiangxi',
 'BeijingChannel',
 'BeijingDiscover',
 'BeijingReview',
 'beltroadnews',
 'bjd_review',
 'BlairZheng1',
 'Bossebosse84',
 'bycongwang',
 'Cao_Li_CHN',
 'CaoYi_MFA',
 'CathayPak',
 'CCGBelfast',
 'CCGJeddah',
 'CCT

In [128]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import chromedriver_binary  

options = Options()
# options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

driver.get('https://twitter.com')
driver.add_cookie(cookie_dict={ 'name': 'auth_token', 'value': '554d238b7147f9fe3fdb32396b8f2e0b73d852f2', 'domain': 'twitter.com' })
driver.get('https://twitter.com')


In [132]:
def get_element(selelctor):
    try:
        element = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, selelctor))
        )
    except Exception as e:
        element = None
    return element

def get_text(element):
    return element.text if element is not None else ''



def scrape(username):
    url = f'https://twitter.com/{username}'
    driver.get(url)

    followers_selector = '#react-root > div > div > div.css-1dbjc4n.r-18u37iz.r-13qz1uu.r-417010 > main > div > div > div > div > div > div:nth-child(3) > div > div > div > div > div.css-1dbjc4n.r-13awgt0.r-18u37iz.r-1w6e6rj > div:nth-child(2) > a > span.css-901oao.css-16my406.r-18jsvk2.r-poiln3.r-1b43r93.r-b88u0q.r-1cwl3u0.r-bcqeeo.r-qvutc0 > span'
    joined_year = '#react-root > div > div > div.css-1dbjc4n.r-18u37iz.r-13qz1uu.r-417010 > main > div > div > div > div > div > div:nth-child(3) > div > div > div > div > div:nth-child(4) > div > span:nth-child(3) > span'
    born_selector = '#react-root > div > div > div.css-1dbjc4n.r-18u37iz.r-13qz1uu.r-417010 > main > div > div > div > div > div > div:nth-child(3) > div > div > div > div > div:nth-child(4) > div > span:nth-child(2)'

    elements = [
        get_element(followers_selector),
        get_element(joined_year),
        get_element(born_selector),
    ]
    
    try:
        profile_image_element = driver.find_element(By.XPATH, '//script[@type="application/ld+json"]').get_attribute('innerHTML')
    # Work with the element
    except NoSuchElementException:
        print(f"suspended {username}")
        return None
    
    follower, joined, born = map(get_text, elements)
    profile_image = json.loads(profile_image_element)
    return (username, follower, joined, profile_image['author']['image']['contentUrl'], born)
    # driver.quit()

In [146]:
import time 

index = 0
for i, key in enumerate(filtered_data):
    if key == 'Guangming_Daily':
        index = i + 1
        break

rest_data = filtered_data[index:]

columns = ('username', 'follwer', 'joined', 'profiled_image', 'born')
with open('../data/x_data5.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(columns)
    cnt = 0


    for username in rest_data:
        scrapped_data = scrape(username)
        if not scrapped_data:
            continue 
        
        time.sleep(2)
        csvwriter.writerow(scrapped_data)
        csvfile.flush()
        cnt += 1

        if cnt == 70:
            cnt = 0
            # It's for avoding rate limit of X.
            time.sleep(60 * 5)
            
    



suspended delkhiintsag
suspended dier0806
suspended Dr_ZhaoQinghua
suspended DrZhaoYanbo
suspended eugenioT14
suspended Frontlinestory
suspended FullFrameCGTN
suspended GeneralkonsulDu


KeyboardInterrupt: 

In [145]:

index = 0
for i, key in enumerate(filtered_data):
    if key == 'Danni_Zhu131':
        index = i
        break

print(filtered_data[index + 1])

delkhiintsag
